In [1]:
import re
import os
import sys
import cv2
import math
import time
import shutil

import pandas as pd
import numpy as np
import pydicom
from utils import *

import matplotlib.pyplot as plt
from tqdm import tqdm
from shutil import copy
from datetime import date
from pydicom.uid import UID, generate_uid
from multiprocessing import dummy as multiprocessing

# 1. video data 학습 위한 csv file 생성

In [2]:
v1_df = pd.read_csv('/home/workspace2/LUS/Dataset/csv_files/version1_v1.csv', index_col=0)
print(len(v1_df))

FileNotFoundError: [Errno 2] No such file or directory: '/home/workspace2/LUS/Dataset/csv_files/version1_v1.csv'

In [29]:
v1_df.head()

,PatientID,Data_idx,StudyDate,Data_type,A-line,B-line,Confluent B-line,Consolidation,Pleural effusion,Lung sliding,Lung Ultrasound Score,study_id
0,24965778,2,20230614,video,NaN,24_32,NaN,NaN,NaN,NaN,1,24965778_2
1,24965778,3,20230614,video,NaN,24_27,NaN,NaN,NaN,NaN,1,24965778_3
2,24965778,4,20230614,video,NaN,NaN,NaN,2_18,NaN,NaN,3_c,24965778_4
3,24965778,5,20230614,video,NaN,NaN,NaN,NaN,1_54,NaN,3_e,24965778_5
4,24965778,6,20230614,video,15_60,NaN,NaN,NaN,NaN,NaN,0,24965778_6


### 이진우 교수님께서 다시 review해야되는 cases 제외
- study_id : 41223316_7, 58262821_3, 76023031_6, 70723403_15, '39009687_11', '39009687_12'

In [30]:
exclusion_std_ids = ['41223316_7', '58262821_3', '76023031_6', '70723403_15', '39009687_11', '39009687_12']

v1_df = v1_df[~v1_df['study_id'].isin(exclusion_std_ids)]

print(len(v1_df))

382


In [31]:
# 해당 열에 대해서 NaN이 아닌 값이 1개인 행을 추출
labels = ['A-line', 'B-line', 'Confluent B-line', 'Consolidation', 'Pleural effusion']
subset_v1_df = v1_df[labels]

# Nan이 아닌 값이 두 개이상 있는 행을 추출하는 조건
mask1 = subset_v1_df.notna().sum(axis=1) == 0
mask1_ = subset_v1_df.notna().sum(axis=1) >= 1

v1_no_label_df = v1_df[mask1]
v1_labeled_df = v1_df[mask1_]

v1_labeled_df = v1_labeled_df.replace('   ', pd.NA)
v1_no_label_df = v1_no_label_df.replace('   ', pd.NA)

print(len(v1_no_label_df))
print(len(v1_labeled_df))

0
382


In [32]:
v1_no_label_df

,PatientID,Data_idx,StudyDate,Data_type,A-line,B-line,Confluent B-line,Consolidation,Pleural effusion,Lung sliding,Lung Ultrasound Score,study_id


In [33]:
v1_labeled_df.head()

,PatientID,Data_idx,StudyDate,Data_type,A-line,B-line,Confluent B-line,Consolidation,Pleural effusion,Lung sliding,Lung Ultrasound Score,study_id
0,24965778,2,20230614,video,NaN,24_32,NaN,NaN,NaN,NaN,1,24965778_2
1,24965778,3,20230614,video,NaN,24_27,NaN,NaN,NaN,NaN,1,24965778_3
2,24965778,4,20230614,video,NaN,NaN,NaN,2_18,NaN,NaN,3_c,24965778_4
3,24965778,5,20230614,video,NaN,NaN,NaN,NaN,1_54,NaN,3_e,24965778_5
4,24965778,6,20230614,video,15_60,NaN,NaN,NaN,NaN,NaN,0,24965778_6


## 1. 제외 case
- align o인데, label이 다른 cases :  39009687_8
- align o, 동일한 label인데 겹치는 영역이 없는 cases : 50030114_36, 50030114_44, 48137795_5, 58262821_9
- 위의 cases에 대해서 version 1에서는 제외

In [34]:
exclusion_std_ids_v1 = ['39009687_8', '50030114_36', '50030114_44', '48137795_5', '58262821_9']

v1_labeled_df = v1_labeled_df[~v1_labeled_df['study_id'].isin(exclusion_std_ids_v1)]

print(len(v1_labeled_df))

377


In [35]:
print(v1_labeled_df.columns)
v1_labeled_df.head()

Index(['PatientID', 'Data_idx', 'StudyDate', 'Data_type', 'A-line', 'B-line',
       'Confluent B-line', 'Consolidation', 'Pleural effusion', 'Lung sliding',
       'Lung Ultrasound Score', 'study_id'],
      dtype='object')


,PatientID,Data_idx,StudyDate,Data_type,A-line,B-line,Confluent B-line,Consolidation,Pleural effusion,Lung sliding,Lung Ultrasound Score,study_id
0,24965778,2,20230614,video,NaN,24_32,NaN,NaN,NaN,NaN,1,24965778_2
1,24965778,3,20230614,video,NaN,24_27,NaN,NaN,NaN,NaN,1,24965778_3
2,24965778,4,20230614,video,NaN,NaN,NaN,2_18,NaN,NaN,3_c,24965778_4
3,24965778,5,20230614,video,NaN,NaN,NaN,NaN,1_54,NaN,3_e,24965778_5
4,24965778,6,20230614,video,15_60,NaN,NaN,NaN,NaN,NaN,0,24965778_6


In [36]:
v1_labeled_df.Data_type.value_counts()

video    341
image     36
Name: Data_type, dtype: int64

In [37]:
video_labeled_df = v1_labeled_df[v1_labeled_df['Data_type'] == 'video']
image_labeled_df = v1_labeled_df[v1_labeled_df['Data_type'] == 'image']
print(len(video_labeled_df))

341


In [38]:
image_labeled_df

,PatientID,Data_idx,StudyDate,Data_type,A-line,B-line,Confluent B-line,Consolidation,Pleural effusion,Lung sliding,Lung Ultrasound Score,study_id
5,24965778,7,20230614,image,1,NaN,NaN,NaN,NaN,NaN,0,24965778_7
6,24965778,8,20230614,image,NaN,1,NaN,NaN,NaN,NaN,1,24965778_8
8,30625107,2,20230720,image,1,NaN,NaN,NaN,NaN,NaN,0,30625107_2
10,30625107,4,20230720,image,NaN,1,NaN,NaN,NaN,NaN,1,30625107_4
13,30625107,7,20230720,image,NaN,NaN,NaN,1,1,NaN,3_ce,30625107_7
14,30625107,8,20230720,image,NaN,NaN,NaN,1,1,NaN,3_ce,30625107_8
23,30625107,17,20230720,image,NaN,NaN,NaN,1,1,NaN,3_ce,30625107_17
26,30625107,20,20230720,image,NaN,NaN,NaN,1,1,NaN,3_ce,30625107_20
30,39009687,4,20230720,image,1,NaN,NaN,NaN,NaN,NaN,0,39009687_4
31,39009687,5,20230720,image,1,NaN,NaN,NaN,NaN,NaN,0,39009687_5


In [39]:
bi_video_labeled_df = video_labeled_df.copy()

# Multi-Label 레이블 생성
label_columns = ["A-line", "B-line", "Confluent B-line", "Consolidation", "Pleural effusion", "Lung sliding"]
for column_name in label_columns:
    bi_video_labeled_df[column_name] = bi_video_labeled_df[column_name].apply(lambda x: 1 if pd.notna(x) else 0)

bi_video_labeled_df

,PatientID,Data_idx,StudyDate,Data_type,A-line,B-line,Confluent B-line,Consolidation,Pleural effusion,Lung sliding,Lung Ultrasound Score,study_id
0,24965778,2,20230614,video,0,1,0,0,0,0,1,24965778_2
1,24965778,3,20230614,video,0,1,0,0,0,0,1,24965778_3
2,24965778,4,20230614,video,0,0,0,1,0,0,3_c,24965778_4
3,24965778,5,20230614,video,0,0,0,0,1,0,3_e,24965778_5
4,24965778,6,20230614,video,1,0,0,0,0,0,0,24965778_6
...,...,...,...,...,...,...,...,...,...,...,...,...
417,57786317,4,20231020,video,0,0,0,1,0,0,3_c,57786317_4
418,57786317,5,20231020,video,0,0,1,0,0,0,2,57786317_5
420,57786317,7,20231020,video,0,0,1,0,0,0,2,57786317_7
421,57786317,8,20231020,video,0,0,1,1,0,0,3_c,57786317_8


In [40]:
len(bi_video_labeled_df.PatientID.value_counts())

35

In [41]:
bi_video_labeled_df.iloc[2]['Lung Ultrasound Score'].split('_')[0]

'3'

In [42]:
bi_video_labeled_df['Lung Ultrasound Score'] = [score.split('_')[0] for score in bi_video_labeled_df['Lung Ultrasound Score']] 
bi_video_labeled_df

,PatientID,Data_idx,StudyDate,Data_type,A-line,B-line,Confluent B-line,Consolidation,Pleural effusion,Lung sliding,Lung Ultrasound Score,study_id
0,24965778,2,20230614,video,0,1,0,0,0,0,1,24965778_2
1,24965778,3,20230614,video,0,1,0,0,0,0,1,24965778_3
2,24965778,4,20230614,video,0,0,0,1,0,0,3,24965778_4
3,24965778,5,20230614,video,0,0,0,0,1,0,3,24965778_5
4,24965778,6,20230614,video,1,0,0,0,0,0,0,24965778_6
...,...,...,...,...,...,...,...,...,...,...,...,...
417,57786317,4,20231020,video,0,0,0,1,0,0,3,57786317_4
418,57786317,5,20231020,video,0,0,1,0,0,0,2,57786317_5
420,57786317,7,20231020,video,0,0,1,0,0,0,2,57786317_7
421,57786317,8,20231020,video,0,0,1,1,0,0,3,57786317_8


# 2. 모든 영상의 video를 dcm -> avi 형식으로 변환

In [43]:
bi_video_labeled_df.head()

,PatientID,Data_idx,StudyDate,Data_type,A-line,B-line,Confluent B-line,Consolidation,Pleural effusion,Lung sliding,Lung Ultrasound Score,study_id
0,24965778,2,20230614,video,0,1,0,0,0,0,1,24965778_2
1,24965778,3,20230614,video,0,1,0,0,0,0,1,24965778_3
2,24965778,4,20230614,video,0,0,0,1,0,0,3,24965778_4
3,24965778,5,20230614,video,0,0,0,0,1,0,3,24965778_5
4,24965778,6,20230614,video,1,0,0,0,0,0,0,24965778_6


In [45]:
def create_path(row):
    root_path = "/home/workspace2/LUS/Dataset/processed_dcm_dataset"
    patient_id = int(row['PatientID'])
    data_idx = f"{row['Data_idx']:05d}"
    return f"{root_path}/{patient_id}/{data_idx}.dcm"

In [47]:
bi_video_labeled_df['dcm_path'] = bi_video_labeled_df.apply(create_path, axis=1)
bi_video_labeled_df.head()

,PatientID,Data_idx,StudyDate,Data_type,A-line,B-line,Confluent B-line,Consolidation,Pleural effusion,Lung sliding,Lung Ultrasound Score,study_id,dcm_path
0,24965778,2,20230614,video,0,1,0,0,0,0,1,24965778_2,/home/workspace2/LUS/Dataset/processed_dcm_dat...
1,24965778,3,20230614,video,0,1,0,0,0,0,1,24965778_3,/home/workspace2/LUS/Dataset/processed_dcm_dat...
2,24965778,4,20230614,video,0,0,0,1,0,0,3,24965778_4,/home/workspace2/LUS/Dataset/processed_dcm_dat...
3,24965778,5,20230614,video,0,0,0,0,1,0,3,24965778_5,/home/workspace2/LUS/Dataset/processed_dcm_dat...
4,24965778,6,20230614,video,1,0,0,0,0,0,0,24965778_6,/home/workspace2/LUS/Dataset/processed_dcm_dat...


In [48]:
bi_video_labeled_df['dcm_path'].iloc[0]

'/home/workspace2/LUS/Dataset/processed_dcm_dataset/24965778/00002.dcm'

In [49]:
def dicom_2_avi(video_path, destinationFolder, folder_name, file_name):

    # fileName = dicom_path.split('/')[-1]

    dataset = pydicom.dcmread(video_path, force=True)
    if dataset.PixelData:

        test_arr = dataset.pixel_array
        if len(test_arr.shape) == 4:  # 동영상인 case만 처리


            # get information to crop each frame
            xmin = dataset[(0x018, 0x6011)].value[0][(0x018, 0x6018)].value
            ymin = dataset[(0x018, 0x6011)].value[0][(0x018, 0x601a)].value
            xmax = dataset[(0x018, 0x6011)].value[0][(0x018, 0x601c)].value
            ymax = dataset[(0x018, 0x6011)].value[0][(0x018, 0x601e)].value

            frames, height, width, channels = test_arr.shape
            cropped_height = ymax-ymin 
            cropped_width = xmax-xmin
            fps = 30

            try:
                fps = dataset[(0x18, 0x40)].value
            except:
                print("couldn't find frame rate, default to 30")

            fourcc = cv2.VideoWriter_fourcc('M', 'J', 'P', 'G')
            video_filename = os.path.join(destinationFolder, folder_name, file_name.replace('.dcm', '') + '.avi')
            out = cv2.VideoWriter(video_filename, fourcc, fps, (cropped_width, cropped_height))
            
            for i in range(frames):

                outputA = test_arr[i, :, :, 0]

                # crop image
                cropped_frame = outputA[ymin:ymax, xmin:xmax]
                
                # if you want resize frame, use it
                # output = cv2.resize(smallOutput, cropSize, interpolation=cv2.INTER_CUBIC)
                
                finaloutput = cv2.merge([cropped_frame, cropped_frame, cropped_frame])
                out.write(finaloutput)
            
            out.release()
        
        else:
            print(f'{file_name} is not a video file.')
            return 0
    else:
        print(f'{file_name} is empty.')
        return 0

In [ ]:
processed_dcm_head_path = '/home/work/kt060_DR-TB/LJH/MICU_LUS/Dataset/processed_dataset_dcm'
destination_folder = '/home/work/kt060_DR-TB/LJH/MICU_LUS/Dataset/processed_dataset_avi'

subfolders = os.listdir(processed_dcm_head_path)
count = 0

for folder in tqdm(subfolders):
    print(folder)

    for file_name in os.listdir(os.path.join(processed_dcm_head_path, folder)):
        count += 1

        video_path = os.path.join(processed_dcm_head_path, folder, file_name)

        # print(count, folder, file_name)

        if not os.path.exists(os.path.join(destination_folder, folder, file_name)):
            os.makedirs(os.path.join(destination_folder, folder), exist_ok=True)
        
        # convert dcm format to avi format
        dicom_2_avi(video_path, destination_folder, folder, file_name)
#         dicom_2_mp4(video_path, destination_folder, folder, file_name)